In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import applications
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
import numpy as np
import glob,os
from scipy.misc import imread,imresize
from keras.models import model_from_json
from os import listdir


### initializations


In [ ]:
trainDirectory = "Data/train"
testDirectory = "Data/test"
imageSize = (224,224)
#imageSize = (299,299)
batchSize = 32
nClasses = 2

### VGG 16 Model

In [ ]:
def load_VGG16_model():
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
  print (" VGG Model loaded..!")
  #print base_model.summary()

  return base_model

In [ ]:
def load_Inception_model():
    base_model = InceptionV3(weights='imagenet', include_top=False)
    print("inception model loaded... ")
    return base_model

### Getting generators 


In [ ]:
def get_test_genertor(dir):
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        dir,
        target_size= imageSize, #    (299, 299),
        batch_size=batchSize ,
        )

    return test_generator


def get_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True,
        rotation_range=10.,
        width_shift_range=0.2,
        height_shift_range=0.2)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        trainDirectory,
        target_size=imageSize, #(299, 299),
        batch_size=batchSize,
        )
        

    validation_generator = test_datagen.flow_from_directory(
        testDirectory,
        target_size= imageSize, #    (299, 299),
        batch_size=batchSize ,
        )

    return train_generator, validation_generator


### Get model for the training


In [ ]:
def GetModel():
    base_model = load_VGG16_model()
    #base_model = load_Inception_model()
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(nClasses, activation='softmax'))    

    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = GetModel()
model.summary()

### Save/Load models

In [ ]:
def SaveModel(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")
def LoadSavedModel():

    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    return loaded_model

### Method for training 

In [ ]:
def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model


### Build the Model and train

In [ ]:
model = GetModel()
generators = get_generators()
model = train_model(model, 10, generators)
#SaveModel(model)

### Or just load from the memory and compile

In [ ]:
model = LoadSavedModel()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
generators = get_generators()
train_generator, validation_generator = generators
acc = model.evaluate_generator(validation_generator, steps=10)

### Evaluate multiple folders (Video-Images)

In [ ]:
testGen = get_test_genertor("Data/test2")
acc = model.evaluate_generator(testGen, steps=10)

In [ ]:
im = imread("Data/test3/Class_1/49H.mp4_37674.jpg")
#im = imread("Data/test/Class_0/v_Diving_g24_c02.avi_138.jpg")
im = imresize(im,(224,224))
im = im.reshape((1,224,224,3))

In [ ]:
def classifyImages(model,dir):
    files = [f for f in listdir(dir) ]
    nFiles = 0
    x = np.zeros((len(files),224,224,3))
    print('Loading files...')
    for f in files:
        fName = "{}{}".format(dir,f)
        #print("Loading {} ...\n".format(f))
        im = imread(fName)
        im = imresize(im,imageSize)
        #x.append(im)
        x[nFiles,:,:,:] = im
    x = np.array(x)
    #x = x.reshape()
    print('Predicting...')
    y = model.predict(x,batch_size=batchSize)
    sumOne = [yIter.argmax() for yIter in y].count(1)
    sumZero = [yIter.argmax() for yIter in y].count(0)
    pred = 'Class_1' if sumOne>sumZero else 'Class_0'
    perc = sumOne/(sumOne+sumZero) if sumOne>sumZero else (sumZero/(sumOne+sumZero))
    return pred,perc

### Calssification result 

In [ ]:
yPred,yPerc = classifyImages(model,"Data/test3/Class_1/")
#files = [f for f in listdir("Data/test3/Class_1/") ]
print(yPred)
print(yPerc)


In [ ]:
base_model = load_VGG16_model()
y = base_model.predict(x)